In [ ]:
#nothing here

In [1]:
import pandas as pd
import requests

In [2]:
api_key='d13943e691d644ba96f4dab8c2e42c09'

In [6]:
url = "https://api.trafikinfo.trafikverket.se/v2/data.json"

query = """
<REQUEST>
  <LOGIN authenticationkey='d13943e691d644ba96f4dab8c2e42c09' />
  <QUERY objecttype="TrainAnnouncement" schemaversion="1.8" orderby="AdvertisedTimeAtLocation">
    <FILTER>
      <!-- Any train that goes via Älvsjö -->
      <EQ name="LocationSignature" value="Äs" />
      <!-- Time window -->
      <GT name="AdvertisedTimeAtLocation" value="2025-09-08T00:00:00.000+02:00" />
      <LT name="AdvertisedTimeAtLocation" value="2025-09-08T23:59:59.000+02:00" />
    </FILTER>
    <INCLUDE>AdvertisedTrainIdent</INCLUDE>
    <INCLUDE>AdvertisedTimeAtLocation</INCLUDE>
    <INCLUDE>EstimatedTimeAtLocation</INCLUDE>
    <INCLUDE>TimeAtLocation</INCLUDE>
    <INCLUDE>LocationSignature</INCLUDE>
    <INCLUDE>TrackAtLocation</INCLUDE>
    <INCLUDE>Advertised</INCLUDE>
  </QUERY>
</REQUEST>
"""
response = requests.post(url, data=query.encode("utf-8"), headers={"Content-Type": "text/xml"})
data = response.json()

In [7]:
data

{'RESPONSE': {'RESULT': [{'TrainAnnouncement': [{'Advertised': True,
      'AdvertisedTimeAtLocation': '2025-09-08T00:03:00.000+02:00',
      'AdvertisedTrainIdent': '2279',
      'LocationSignature': 'Äs',
      'TimeAtLocation': '2025-09-08T00:03:00.000+02:00',
      'TrackAtLocation': '4'},
     {'Advertised': True,
      'AdvertisedTimeAtLocation': '2025-09-08T00:03:00.000+02:00',
      'AdvertisedTrainIdent': '2279',
      'LocationSignature': 'Äs',
      'TimeAtLocation': '2025-09-08T00:02:00.000+02:00',
      'TrackAtLocation': '4'},
     {'Advertised': True,
      'AdvertisedTimeAtLocation': '2025-09-08T00:12:00.000+02:00',
      'AdvertisedTrainIdent': '2980',
      'LocationSignature': 'Äs',
      'TimeAtLocation': '2025-09-08T00:10:00.000+02:00',
      'TrackAtLocation': '3'},
     {'Advertised': True,
      'AdvertisedTimeAtLocation': '2025-09-08T00:12:00.000+02:00',
      'AdvertisedTrainIdent': '2980',
      'LocationSignature': 'Äs',
      'TimeAtLocation': '2025-09-08T0